In [1]:
import requests
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
# Step 2: Data Retrieval
def fetch_stock_data(symbol, api_key):
    base_url = "https://www.alphavantage.co/query"
    function = "TIME_SERIES_DAILY"
    output_size = "full"

    params = {
        "function": function,
        "symbol": symbol,
        "outputsize": output_size,
        "apikey": api_key
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if "Time Series (Daily)" in data:
        return data["Time Series (Daily)"]
    else:
        print("Error retrieving stock data.")

# Step 3: Data Storage
def store_stock_data(stock_data):
    df = pd.DataFrame(stock_data).T
    df.index = pd.to_datetime(df.index)
    df = df.astype(float)
    return df

# Step 4: Statistical Analysis
def calculate_statistics(df):
    returns = df['close'].pct_change().dropna()
    mean_return = returns.mean()
    return {
        'mean_return': mean_return,
        'annualized_return': (1 + mean_return) ** 252 - 1,
        'volatility': returns.std() * np.sqrt(252),
        'sharpe_ratio': mean_return / (returns.std() * np.sqrt(252))
    }

# Step 5: Portfolio Optimization
def calculate_portfolio(weights, returns, cov_matrix):
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

def optimize_portfolio(returns, cov_matrix):
    num_assets = len(returns)
    args = (returns, cov_matrix)

    def objective(weights):
        return -calculate_portfolio(weights, returns, cov_matrix)[0]  # Maximize return

    constraints = (
        {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # Weights sum to 1
        {'type': 'ineq', 'fun': lambda x: x}  # No short-selling
    )
    bounds = tuple((0, 1) for _ in range(num_assets))
    initial_weights = num_assets * [1 / num_assets]  # Equally weighted initial portfolio

    result = minimize(objective, initial_weights, args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Step 6: Complex Data Visualization
def plot_stock_data(df, moving_average):
    # ... Code from the previous example ...

# Step 6: Complex Data Visualization
def plot_efficient_frontier(returns, cov_matrix, num_portfolios=1000):
    port_returns = []
    port_volatility = []
    port_weights = []

    for _ in range(num_portfolios):
        weights = np.random.random(len(returns))
        weights /= np.sum(weights)

        portfolio_return, portfolio_volatility = calculate_portfolio(weights, returns, cov_matrix)

        port_returns.append(portfolio_return)
        port_volatility.append(portfolio_volatility)
        port_weights.append(weights)

    port_returns = np.array(port_returns)
    port_volatility = np.array(port_volatility)

    # Plot efficient frontier
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=port_volatility,
        y=port_returns,
        mode='markers',
        marker=dict(
            size=6,
            color=port_returns,
            colorscale='Viridis',
            showscale=True
        ),
        name='Portfolios'
    ))
    fig.update_layout(
        title='Efficient Frontier',
        xaxis_title='Volatility',
        yaxis_title='Return',
        showlegend=False
    )
    fig.show()

# Example usage
stock_symbol = "AAPL"  # Apple Inc.
api_key = "YOUR_API_KEY"  # Replace with your Alpha Vantage API key

stock_data = fetch_stock_data(stock_symbol, api_key)
df = store_stock_data(stock_data)
moving_average = calculate_moving_average(df, 20)

# Step 4: Statistical Analysis
statistics = calculate_statistics(df)
print('Mean Return:', statistics['mean_return'])
print('Annualized Return:', statistics['annualized_return'])
print('Volatility:', statistics['volatility'])
print('Sharpe Ratio:', statistics['sharpe_ratio'])

# Step 5: Portfolio Optimization
returns = df.pct_change().dropna()
cov_matrix = returns.cov()
optimized_weights = optimize_portfolio(returns.mean(), cov_matrix)
print('Optimized Weights:', optimized_weights)

# Step 6: Complex Data Visualization
plot_stock_data(df, moving_average)
plot_efficient_frontier(returns.mean(), cov_matrix)